모듈 ∙ 라이브러리 import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')
# tf.config.experimental.list_physical_devices('GPU')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

데이터 불러오기 ∙ 전처리

In [2]:
def filter_df(df): #데이터프레임 필터링
    # 첫 번째 열에서 같은 값을 가진 행의 수를 계산합니다.
    row_counts = df['0'].value_counts()

    # 가장 많은 행의 수를 찾습니다.
    max_row_count = row_counts.max()

    # 가장 많은 행의 수에 해당하는 행만 분류합니다.
    filtered = pd.DataFrame(df[df['0'].isin(row_counts[row_counts == max_row_count].index)])

    return filtered

# 데이터 로드   
stock_df = pd.read_csv('/Users/moon/Desktop/Moon SeungHoo/Stock_Machine_Learning/StockData_5%.csv',low_memory=False)

#데이터 필터링
filter_stock = filter_df(stock_df)
filter_label = filter_stock['24']

# 불필요한 데이터 삭제
filter_stock = filter_stock.drop({'0','1','7','24'},axis=1) #날자, 상승율, 5%이상 상승여부 삭제 

데이터 전처리

In [3]:
scaler = MinMaxScaler()
stock_label = scaler.fit_transform(filter_label.values.reshape(-1, 1))

count_zeros = np.sum(stock_label == 0)
count_ones = np.sum(stock_label == 1)
total_samples = len(stock_label)

ratio_zeros = (count_zeros / total_samples).round(2)
ratio_ones = (count_ones / total_samples).round(2)

print("Ratio of zeros (0s):", ratio_zeros)
print("Ratio of ones (1s):", ratio_ones)

Ratio of zeros (0s): 0.96
Ratio of ones (1s): 0.04


데이터 분할

In [4]:
# 훈련 및 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(filter_stock, stock_label, test_size=0.2, random_state=42)

smote = SMOTE(sampling_strategy=1 ,random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train,y_train)
X_test_resampled, y_test_resampled = smote.fit_resample(X_test,y_test)

count_zeros = np.sum(y_train_resampled == 0)
count_ones = np.sum(y_train_resampled == 1)
total_samples = len(y_train_resampled)

ratio_zeros = (count_zeros / total_samples).round(2)
ratio_ones = (count_ones / total_samples).round(2)

print("Ratio of zeros (0s):", ratio_zeros)
print("Ratio of ones (1s):", ratio_ones)

Ratio of zeros (0s): 0.5
Ratio of ones (1s): 0.5


GRU 모델 정의

In [5]:
# GRU 모델 정의
model = tf.keras.Sequential()
model.add(tf.keras.layers.GRU(18, input_shape=(X_train_resampled.shape[1],1),return_sequences=True))
model.add(tf.keras.layers.GRU(18, return_sequences=True))
model.add(tf.keras.layers.GRU(18, return_sequences=True))
model.add(tf.keras.layers.GRU(18, return_sequences=True))
model.add(tf.keras.layers.GRU(18, return_sequences=True))
model.add(tf.keras.layers.GRU(18, return_sequences=True))
model.add(tf.keras.layers.GRU(18, return_sequences=True))
model.add(tf.keras.layers.GRU(18, return_sequences=True))
model.add(tf.keras.layers.GRU(18, return_sequences=False))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 조기 종료 콜백 정의
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 28, 18)            1134      
                                                                 
 gru_1 (GRU)                 (None, 28, 18)            2052      
                                                                 
 gru_2 (GRU)                 (None, 28, 18)            2052      
                                                                 
 gru_3 (GRU)                 (None, 28, 18)            2052      
                                                                 
 gru_4 (GRU)                 (None, 28, 18)            2052      
                                                                 
 gru_5 (GRU)                 (None, 28, 18)            2052      
                                                                 
 gru_6 (GRU)                 (None, 28, 18)            2

모델 훈련

In [6]:
# 모델 훈련
model.fit(X_train_resampled,y_train_resampled,epochs=500,
validation_data=(X_test_resampled,y_test_resampled),callbacks=[early_stopping])
 
# 모델 저장

# model.save("GRU_Model.h5")3  
model.save("GRU_Model_9L_18.h5")

Epoch 1/500
28930/28930 [==============================] - 667s 23ms/step - loss: 0.2801 - accuracy: 0.8814 - val_loss: 0.2368 - val_accuracy: 0.9045
Epoch 2/500
28930/28930 [==============================] - 654s 23ms/step - loss: 0.2397 - accuracy: 0.9011 - val_loss: 0.2456 - val_accuracy: 0.8974
Epoch 3/500
28930/28930 [==============================] - 661s 23ms/step - loss: 0.2342 - accuracy: 0.9035 - val_loss: 0.2257 - val_accuracy: 0.9080
Epoch 4/500
28930/28930 [==============================] - 674s 23ms/step - loss: 0.2267 - accuracy: 0.9069 - val_loss: 0.2226 - val_accuracy: 0.9095
Epoch 5/500
28930/28930 [==============================] - 672s 23ms/step - loss: 0.2256 - accuracy: 0.9078 - val_loss: 0.2229 - val_accuracy: 0.9097
Epoch 6/500
28930/28930 [==============================] - 690s 24ms/step - loss: 0.2241 - accuracy: 0.9087 - val_loss: 0.2245 - val_accuracy: 0.9089
Epoch 7/500
28930/28930 [==============================] - 684s 24ms/step - loss: 0.2203 - accuracy:

/Users/moon/anaconda3/envs/ML/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


모델 평가

In [7]:
# 모델 로드
loaded_model = tf.keras.models.load_model("GRU_Model_9L_18.h5")

# 모델을 사용하여 주가 상승 여부 예측
test = loaded_model.predict(X_test)

3771/3771 [==============================] - 21s 5ms/step


In [8]:
y_pred_binary = (test > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_binary)
conf_matrix = confusion_matrix(y_test, y_pred_binary)
classification_rep = classification_report(y_test, y_pred_binary)

print(f"Accuracy: {accuracy}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(classification_rep)

Accuracy: 0.8908694643330681
Confusion Matrix:
[[103092  12646]
 [   523   4411]]
Classification Report:
              precision    recall  f1-score   support

         0.0       0.99      0.89      0.94    115738
         1.0       0.26      0.89      0.40      4934

    accuracy                           0.89    120672
   macro avg       0.63      0.89      0.67    120672
weighted avg       0.96      0.89      0.92    120672



In [9]:
# 예측 결과를 이진값(0 또는 1)으로 변환
binary_predictions = (test > 0.5).astype(int)

# 예측 결과 출력
correct_predictions = np.equal(binary_predictions, y_test)  # 정확하게 예측한 경우 True, 그렇지 않으면 False
accuracy = np.mean(correct_predictions)  # 정확도 계산

print(f"정확도: {accuracy * 100:.2f}%")

정확도: 89.09%
